In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import pandas as pd
import numpy as np
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [0]:
import pickle
import pandas as pd
data = pd.read_pickle('train.pkl')

In [0]:
data.head()

,Id,c,r,y
0,0,_t_ comment faire pour choisir bonne musique ...,merci,1
1,1,_t_ comment faire pour choisir bonne musique ...,لباس العربي,0
2,2,_t_ comment faire pour choisir bonne musique ...,mkber 9albk,0
3,3,_t_ comment faire pour choisir bonne musique ...,thats kayle taswira mouneka ommha check sword...,0
4,4,_t_ comment faire pour choisir bonne musique ...,صحيح تتعداش أبداا dinar,0


In [0]:
data['r_c'] = data['c'] + "--" + data['r']
data = data.drop(columns=['r', 'c', 'Id'])

In [0]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85000 entries, 0 to 84999
Data columns (total 2 columns):
y      85000 non-null int64
r_c    85000 non-null object
dtypes: int64(1), object(1)
memory usage: 1.3+ MB


In [0]:
data = data.to_numpy()

In [0]:
data

array([[1,
        ' _t_ comment faire pour choisir bonne musique pour samma3ni _t_ Bonsoir Rimo simple consultez catalogue toutes tonalités Samma3ni lien suivant: https://samma3ni.ooredoo.tn/oortn/front/# /web/home votre service Lotfi Community Manager _t_ Merci peux donner musique numero _t_ vous cherchez titre bien déterminé Lotfi Community Manager _t_ connais jara _t_ Vous pouvez faire recherche avec titre composant *150*4#. Lotfi Community Manager _t_ Vous dites suspendu solde _t_ Dans vous devez recharger votre compte pour vous piussez acheter tonalité. votre service Lotfi Community Manager--merci'],
       [0,
        ' _t_ comment faire pour choisir bonne musique pour samma3ni _t_ Bonsoir Rimo simple consultez catalogue toutes tonalités Samma3ni lien suivant: https://samma3ni.ooredoo.tn/oortn/front/# /web/home votre service Lotfi Community Manager _t_ Merci peux donner musique numero _t_ vous cherchez titre bien déterminé Lotfi Community Manager _t_ connais jara _t_ Vous pouvez

In [0]:
X=data[:,1]
y=data[:,0]

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [0]:
total = np.concatenate((X_test, X_train), axis=0)

In [0]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(total)

In [0]:
max_length=max([len(s.split()) for s in total])

In [0]:
vocab_size=len(tokenizer.word_index)+1
X_train=tokenizer.texts_to_sequences(X_train)
X_test=tokenizer.texts_to_sequences(X_test)

In [0]:
X_trainp=pad_sequences(X_train , maxlen=max_length, padding='post')
X_testp=pad_sequences(X_test , maxlen=max_length, padding='post')

In [0]:
X_testp

array([[    1,  1514,   302, ...,     0,     0,     0],
       [    1,  1759,  7924, ...,     0,     0,     0],
       [    1,  2857,    31, ...,     0,     0,     0],
       ...,
       [    1, 12608,  5526, ...,     0,     0,     0],
       [    1,   226,  2314, ...,     0,     0,     0],
       [    1,   179,   195, ...,     0,     0,     0]], dtype=int32)

In [0]:
X_trainp

array([[   1, 5437, 1569, ...,    0,    0,    0],
       [   1, 1204,  260, ...,    0,    0,    0],
       [   1,   95,   22, ...,    0,    0,    0],
       ...,
       [   1,  118, 3914, ...,    0,    0,    0],
       [   1,   22,  128, ...,    0,    0,    0],
       [   1,   20,   84, ...,    0,    0,    0]], dtype=int32)

In [0]:
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
history = model.fit(X_trainp,y_train, epochs=10,
                    validation_data=(X_testp,y_test), 
                    validation_steps=10, batch_size=128)

Train on 68000 samples, validate on 17000 samples
Epoch 1/10
68000/68000 [==============================] - 1722s 25ms/sample - loss: 0.3565 - acc: 0.9004 - val_loss: 0.3212 - val_acc: 0.8984
Epoch 2/10
68000/68000 [==============================] - 1919s 28ms/sample - loss: 0.2985 - acc: 0.9004 - val_loss: 0.3073 - val_acc: 0.8984
Epoch 3/10
68000/68000 [==============================] - 2186s 32ms/sample - loss: 0.2689 - acc: 0.9009 - val_loss: 0.2988 - val_acc: 0.8954
Epoch 4/10
68000/68000 [==============================] - 2538s 37ms/sample - loss: 0.2389 - acc: 0.9053 - val_loss: 0.2976 - val_acc: 0.8934
Epoch 5/10
67968/68000 [============================>.] - ETA: 0s - loss: 0.2087 - acc: 0.9150

In [0]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string], '')
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()